In [235]:
# Import libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import string

In [236]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [237]:
# Download required NLTK data
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab') # Download the punkt_tab resource

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [238]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng') # Download the specific 'eng' resource


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [239]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [240]:
# Import libraries
import pandas as pd
import nltk
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle

# Download necessary NLTK corpora
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [241]:
df = pd.read_excel('/content/Book1.xlsx')

In [242]:
# Drop rows with missing values
df.dropna(subset=['review', 'sentiment'], inplace=True)

In [243]:
# Balance the dataset (equal number of samples per class)
min_class = df['sentiment'].value_counts().min()
df_balanced = df.groupby('sentiment').sample(n=min_class, random_state=42)
df_balanced = shuffle(df_balanced, random_state=42).reset_index(drop=True)

In [244]:
# Convert text to lowercase
df_balanced['review'] = df_balanced['review'].str.lower()

In [245]:
# Remove punctuation
df_balanced['review'] = df_balanced['review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [246]:
# Tokenize
df_balanced['review'] = df_balanced['review'].apply(word_tokenize)

In [247]:
# Remove stopwords (but keep 'not')
stop_words = set(stopwords.words('english'))
stop_words.discard('not')
df_balanced['review'] = df_balanced['review'].apply(lambda x: ' '.join([word for word in x if word not in stop_words]))

In [248]:
# Lemmatization with POS tagging
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return 'n'

def lemmatize_with_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags])

df_balanced['review'] = df_balanced['review'].apply(lemmatize_with_pos)

In [249]:
# Feature extraction
vectorizer = CountVectorizer(max_features=2000)
X = vectorizer.fit_transform(df_balanced['review'])
y = df_balanced['sentiment']

In [250]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [251]:
# Model training
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [252]:
# Evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9666666666666667
Classification Report:
               precision    recall  f1-score   support

    negative       0.98      0.95      0.96        55
     neutral       1.00      0.96      0.98        49
    positive       0.92      1.00      0.96        46

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150



In [253]:
# Function to clean and predict new review
def preprocess_and_predict(review_text):
    review_text = review_text.lower()
    review_text = review_text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(review_text)
    tokens = [word for word in tokens if word not in stop_words]
    review_text = ' '.join(tokens)
    review_text = lemmatize_with_pos(review_text)
    features = vectorizer.transform([review_text])
    prediction = model.predict(features)[0]
    return prediction


In [254]:
# Get input review from user
new_review = input("Enter your review: ")
predicted_sentiment = preprocess_and_predict(new_review)
print("Predicted Sentiment:", predicted_sentiment)

Enter your review: i can not use it
Predicted Sentiment: negative


In [257]:
import os

# Create the directory if it doesn't exist
if not os.path.exists('/mnt/data'):
    os.makedirs('/mnt/data')

# Now you can save the DataFrame
df_original.to_csv('/mnt/data/autoinsight_reviews_with_prediction.csv', index=False)
print("New review saved to 'autoinsight_reviews_with_prediction.csv'")

New review saved to 'autoinsight_reviews_with_prediction.csv'
